# Analyse et Nettoyage du dataset UE

Seuls les véhicules des années 2022-2023 et de la France ont été exportés à partir de https://www.eea.europa.eu/data-and-maps/data/co2-cars-emission-20

# <font color='#3585CD'>Importation des librairies</font>

In [1]:
import warnings
warnings.filterwarnings('ignore')
warnings.warn('DelftStack')
warnings.warn('Do not show this message')

import pandas as pd

import numpy as np

import matplotlib.pyplot as plt

import seaborn as sns

import plotly.figure_factory as ff
import plotly.express as px
import plotly.graph_objects as go

import kagglehub

from scipy.stats import gaussian_kde

import sys
import os

# <font color='#3585CD'>Import des fonctions qui seront utilisées dans ce notebook</font>

In [2]:
# On vérifie si on est sur Google Colab
try:
    import google.colab
    ON_COLAB = True
except ImportError:
    ON_COLAB = False

# Gestion de l'import de utils.py
if ON_COLAB:
    from google.colab import drive

    # On monte Google Drive
    drive.mount('/content/drive')

    # On ajoute le dossier contenant utils.py au chemin
    utils_path = "/content/drive/My Drive/Formation DS/Projet CO2/GitHub/NOV24-CDS-CO2/notebooks/utils/"
    sys.path.append(utils_path)

else:
    # Si en local, le fichier utils.py doit être dans le même dossier ou un sous-dossier
    utils_path = os.path.abspath("./utils/")  # Chemin du dossier actuel
    sys.path.append(utils_path)

# Vérifier si le fichier utils.py existe avant import
if os.path.exists(os.path.join(utils_path, "utils.py")):
    import utils
    print("utils.py importé avec succès")
else:
    print("ERREUR : utils.py introuvable, vérifiez le chemin !")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
utils.py importé avec succès


# <font color='#3585CD'>Chargement des données</font>

## Chargement du dataset principal

In [3]:
# Le dataset étant volumineux pour être dans le repository Github, nous le téléchargeons via Kaggle test
path = kagglehub.dataset_download("dimitrileloup/vehicules-fr-2022-2023")
print("Chemin vers le fichier : ", path)

Chemin vers le fichier :  /root/.cache/kagglehub/datasets/dimitrileloup/vehicules-fr-2022-2023/versions/1


In [4]:
dataset_path = f"{path}/datas_FR_2022_2023.csv"
df = pd.read_csv(dataset_path)
df.head(10)

,ID,Country,VFN,Mp,Mh,Man,MMS,Tan,T,Va,...,Erwltp (g/km),De,Vf,Status,year,Date of registration,Fuel consumption,ech,RLFI,Electric range (km)
0,76627138,FR,IP-03_BU_0214-1C4-1,STELLANTIS,CHRYSLER,FCA US LLC,NaN,e3*2007/46*0300*33,BU,AXT23,...,NaN,NaN,NaN,F,2022,2022-01-25,2.0,NaN,NaN,43.0
1,76627139,FR,IP-03_BU_0214-1C4-1,STELLANTIS,CHRYSLER,FCA US LLC,NaN,e3*2007/46*0300*32,BU,AXT23,...,NaN,NaN,NaN,F,2022,2022-02-16,2.0,NaN,NaN,43.0
2,76627140,FR,IP-03_BU_0214-1C4-1,STELLANTIS,CHRYSLER,FCA US LLC,NaN,e3*2007/46*0300*33,BU,AXT23,...,NaN,NaN,NaN,F,2022,2022-02-24,2.0,NaN,NaN,43.0
3,76627141,FR,IP-03_BU_0214-1C4-1,STELLANTIS,CHRYSLER,FCA US LLC,NaN,e3*2007/46*0300*33,BU,AXT23,...,NaN,NaN,NaN,F,2022,2022-02-28,2.0,NaN,NaN,43.0
4,76627142,FR,IP-03_BU_0214-1C4-1,STELLANTIS,CHRYSLER,FCA US LLC,NaN,e3*2007/46*0300*33,BU,AXT23,...,NaN,NaN,NaN,F,2022,2022-01-28,2.0,NaN,NaN,43.0
5,76627143,FR,IP-03_BU_0214-1C4-1,STELLANTIS,CHRYSLER,FCA US LLC,NaN,e3*2007/46*0300*33,BU,AXT23,...,NaN,NaN,NaN,F,2022,2022-04-21,2.0,NaN,NaN,43.0
6,76627144,FR,IP-03_BU_0214-1C4-1,STELLANTIS,CHRYSLER,FCA US LLC,NaN,e3*2007/46*0300*32,BU,AXT23,...,NaN,NaN,NaN,F,2022,2022-01-28,2.0,NaN,NaN,43.0
7,76627145,FR,IP-03_BU_0214-1C4-1,STELLANTIS,CHRYSLER,FCA US LLC,NaN,e3*2007/46*0300*33,BU,AXT23,...,NaN,NaN,NaN,F,2022,2022-04-21,2.0,NaN,NaN,43.0
8,76627146,FR,IP-03_BU_0214-1C4-1,STELLANTIS,CHRYSLER,FCA US LLC,NaN,e3*2007/46*0300*33,BU,AXT23,...,NaN,NaN,NaN,F,2022,2022-04-22,2.0,NaN,NaN,43.0
9,76627147,FR,IP-03_BU_0214-1C4-1,STELLANTIS,CHRYSLER,FCA US LLC,NaN,e3*2007/46*0300*32,BU,AXT23,...,NaN,NaN,NaN,F,2022,2022-01-31,2.0,NaN,NaN,43.0


In [5]:
# Suppression des espaces accidentels dans les noms des colonnes comme "Fuel consumption "
df.columns = df.columns.str.strip()

## Chargement du dataset du dictionnaire des variables

In [6]:
path_vars = kagglehub.dataset_download("dimitrileloup/dfinition-des-colonnes")
print("Path to dataset files:", path_vars)

Path to dataset files: /root/.cache/kagglehub/datasets/dimitrileloup/dfinition-des-colonnes/versions/1


In [7]:
pd.set_option('display.max_colwidth', None) # pour pouvoir afficher tout le descriptif
dataset_variables = f"{path_vars}/Table-definition.xlsx"
var = pd.read_excel(dataset_variables)
var.head(40)

,Nom de la variable,Descriptif,Type
0,ID,Identifiant,int64
1,Country,Pays,object
2,VFN,Numéro d'identification de la famille du véhicule.,object
3,Mp,Pool des constructeurs,object
4,Mh,Nom du fabricant Dénomination standard de l'UE.,object
5,Man,Déclaration OEM du nom du fabricant.,object
6,MMS,Nom du fabricant Dénomination du registre MS.,float64
7,Tan,Numéro d'homologation de type.,object
8,T,Type,object
9,Va,Variante,object


# <font color='#3585CD'>Premières analyses</font>

## Informations sur le dataset

In [8]:
print("\nAperçu du dataset :")
print(df.info())


Aperçu du dataset :
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3528480 entries, 0 to 3528479
Data columns (total 40 columns):
 #   Column                Dtype  
---  ------                -----  
 0   ID                    int64  
 1   Country               object 
 2   VFN                   object 
 3   Mp                    object 
 4   Mh                    object 
 5   Man                   object 
 6   MMS                   float64
 7   Tan                   object 
 8   T                     object 
 9   Va                    object 
 10  Ve                    object 
 11  Mk                    object 
 12  Cn                    object 
 13  Ct                    object 
 14  Cr                    object 
 15  r                     int64  
 16  m (kg)                float64
 17  Mt                    float64
 18  Enedc (g/km)          float64
 19  Ewltp (g/km)          int64  
 20  W (mm)                float64
 21  At1 (mm)              float64
 22  At2 (mm)             

## Satistiques descriptives

In [9]:
print("\nStatistiques descriptives :")
display(df.describe(include='number').T)


Statistiques descriptives :


,count,mean,std,min,25%,50%,75%,max
ID,3528480.0,1.013037e+08,2.222407e+07,2281.0,77509525.75,1.211759e+08,1.220580e+08,1.229402e+08
MMS,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
r,3528480.0,1.000000e+00,0.000000e+00,1.0,1.00,1.000000e+00,1.000000e+00,1.000000e+00
m (kg),3528479.0,1.433451e+03,3.009387e+02,570.0,1197.00,1.367000e+03,1.603000e+03,2.965000e+03
Mt,3528479.0,1.547239e+03,3.138880e+02,616.0,1303.00,1.481000e+03,1.732000e+03,3.150000e+03
Enedc (g/km),284171.0,9.175725e+00,1.574223e+01,0.0,0.00,0.000000e+00,2.700000e+01,4.900000e+01
Ewltp (g/km),3528480.0,9.972159e+01,5.134716e+01,0.0,96.00,1.200000e+02,1.320000e+02,5.720000e+02
W (mm),1638878.0,2.634966e+03,1.318641e+02,1873.0,2552.00,2.605000e+03,2.680000e+03,3.665000e+03
At1 (mm),1638878.0,1.543811e+03,5.045288e+01,1220.0,1513.00,1.550000e+03,1.575000e+03,1.776000e+03
At2 (mm),1637695.0,1.541500e+03,5.550988e+01,1301.0,1506.00,1.542000e+03,1.580000e+03,1.758000e+03


In [10]:
df.describe(include="object").T

,count,unique,top,freq
Country,3528480,1,FR,3528480
VFN,3528474,3084,IP-JAA1MTPJT4A_000-VF1-1,78050
Mp,3326798,14,STELLANTIS,1050427
Mh,3528480,81,RENAULT,637662
Man,3528480,80,RENAULT SAS,637719
Tan,3528479,3636,e2*2007/46*0639*20,55694
T,3528479,448,U,391555
Va,3528479,2156,P,190019
Ve,3528478,9620,V/AMXS,37623
Mk,3528480,80,RENAULT,564592


## Nombre de valeurs uniques par colonne

In [11]:
# Calculer le nombre de valeurs uniques par colonne et renommer la colonne
df_unique_values = df.nunique().sort_values(ascending=False).reset_index()
df_unique_values.columns = ["colonne", "nombre de valeurs uniques"]

df_unique_values

,colonne,nombre de valeurs uniques
0,ID,3528480
1,Ve,9620
2,Tan,3636
3,VFN,3084
4,Va,2156
5,Mt,2070
6,m (kg),1276
7,Cn,1266
8,Date of registration,715
9,Electric range (km),587


## Analyse rapide des colonnes
<p>La fonction analyse_columns permet de visualiser les différentes colonnes avec un échantillon de valeurs</p>

In [12]:
analysis_df = utils.analyse_columns(df)
analysis_df

,Nom de la colonne,Type de la colonne,Nombre de valeurs uniques,Exemples de valeurs
0,ID,int64,3528480,76627138 | 76627139 | 76627140 | 76627141 | 76627142
1,Country,object,1,FR
2,VFN,object,3084,IP-03_BU_0214-1C4-1 | IP-E4JLESSSPOSAH-1C4-1 | IP-E4JLESSRUB-1C4-1 | IP-E4JLPHEVSAH-1C4-1 | IP-e4JLPHEVSAH-1C4-1
3,Mp,object,14,STELLANTIS | TESLA-HONDA-JLR | FORD | MAZDA-SUBARU-SUZUKI-TOYOTA | RENAULT-NISSAN-MITSUBISHI
4,Mh,object,81,CHRYSLER | HONDA MOTOR CO | FORD MOTOR COMPANY | GENERAL MOTORS HOLDINGS | MAZDA
5,Man,object,80,FCA US LLC | HONDA MOTOR CO LTD | FORD MOTOR COMPANY | GENERAL MOTORS HOLDINGS LLC | MAZDA MOTOR CORPORATION
6,MMS,float64,0,
7,Tan,object,3636,e3*2007/46*0300*33 | e3*2007/46*0300*32 | e4*2001/116*0116*38 | e4*2001/116*0116*40 | e4*2001/116*0116*41
8,T,object,448,BU | JK | RV | LAE | WAH
9,Va,object,2156,AXT23 | JSLFZ | JTAFG | RV501 | AXN1B


# <font color='#3585CD'>Analyse des valeurs manquantes</font>

In [13]:
data_na = utils.display_missing_values(df)
data_na

,Colonne,Valeurs manquantes (%),Nombre de valeurs manquantes,Type
0,MMS,100.000000,3528480,float64
1,Vf,100.000000,3528480,float64
2,Ernedc (g/km),100.000000,3528480,float64
3,De,100.000000,3528480,float64
4,RLFI,98.216456,3465548,object
5,Enedc (g/km),91.946362,3244309,float64
6,z (Wh/km),76.950273,2715175,float64
7,Electric range (km),76.950273,2715175,float64
8,At2 (mm),53.586388,1890785,float64
9,W (mm),53.552861,1889602,float64


# <font color='#3585CD'>Suppression de colonnes</font>

Nous pouvons dès à présent ces colonnes qui ont un taux de valeurs manquantes supérieur à 70% :

*   At2 (mm)
*   W (mm)
*   MMS
*   Vf
*   De
*   Ernedc (g/km)
*   At1 (mm)
*   Enedc (g/km)
*   RLFI
*   z (Wh/km)
*   Electric range (km)

In [14]:
df = df.drop(columns=['At2 (mm)', 'W (mm)', 'MMS', 'Vf', 'De', 'Ernedc (g/km)',	'At1 (mm)',	'Enedc (g/km)',	'RLFI',	'z (Wh/km)', 'Electric range (km)'], axis=1)

In [ ]:
df.duplicated().sum()

In [ ]:
data_na = utils.display_missing_values(df)
data_na

# <font color='#3585CD'>Suppresion des colonnes non pertinentes</font>

Certaines colonnes n'ont pas d'intérêt à être gardées :

*   IT
*   Erwltp (g/km) (dépreciée)
*   ID : identifiant du véhicule
*   Country : notre dataset est une extraction des véhicules de France
*   VFN : n'a pas de norme universelle et comporte trop de valeurs
*   Tan : trop de valeurs et sans intérêt pour notre projet
*   T : trop de valeurs et sans intérêt pour notre projet
*   Va : trop de valeurs et sans intérêt pour notre projet
*   Ve : trop de valeurs et sans intérêt pour notre projet
*   Status : n'a qu'une seule valeur et ne varie pas
*   Year : 1 seule valeur
*   Date of registration : sans intérêt pour notre projet
*   Fm : redondant avec Ft
*   Cr : nous avons 2 catégories (M1, M1G). M1G est une sous-catégorie de M1 réservée aux véhicules tout-terrain. Nous pouvons conclure que tous les véhicules sont de catégorie M1
*   Ct : idem que Cr
*   ech : sans intérêt pour notre projet
*   Mp : redondant, se retrouve dans une autre colonne
*   Man : redondant avec Mk
*   r : n'a qu'une seule valeur
*   Mh : redondant avec Mk


In [ ]:
df = df.drop(columns=['IT', 'Erwltp (g/km)', 'ID', 'Country', 'VFN', 'Tan', 'T', 'Va', 'Ve', 'Status', 'year', 'Date of registration', 'Fm', 'Cr', 'Ct', 'ech', 'Mp', 'Man', 'r', 'Mh'], axis=1)

## Vérification de la corrélation entre Masse à vide et Masse totale</font>

In [ ]:
df_masse = df[['Mt', 'm (kg)']]
utils.plot_correlation_matrix(df_masse)

On se rend compte qu'il y a une **forte corrélataion** entre ces 2 variables, qui pourrait entrainer une **colinéarité**. Nous prenons la décision de ne garder que la masse à vide (m (kg))

In [ ]:
df = df.drop('Mt', axis=1)

# <font color='#3585CD'>Gestion des doublons</font>

## Nombre de doublons

In [ ]:
df.duplicated().sum()

## Supression des doublons

In [ ]:
df.drop_duplicates(inplace =True)

In [ ]:
df.duplicated().sum()

In [ ]:
df.shape

In [ ]:
df = df.reset_index(drop=True)

# <font color='#3585CD'>Renommage des colonnes</font>

Pour plus de compréhension, nous allons renommer les colonnes :



*   Mk : Marque
*   Cn : Modele
*   Ewltp (g/km) : Co2
*   Ft : Carburant
*   ec (cm3) : Cylindree moteur
*   ep (KW) : Puissance moteur
*   Fuel consumption : Consommation carburant


In [ ]:
renommage = {
    'Mk': 'Marque',
    'Cn': 'Modèle',
    'm (kg)' : 'Masse à vide',
    'Ewltp (g/km)': 'CO2',
    'Ft': 'Carburant',
    'ec (cm3)': 'Cylindrée moteur',
    'ep (KW)': 'Puissance moteur',
    'Fuel consumption': 'Consommation carburant'
}

# Application du renommage
df.rename(columns=renommage, inplace=True)

In [ ]:
df

# <font color='#3585CD'>Faut-il garder les véhicules électriques et hydrogènes ?</font>
Notre objectif est de prédire les émissions directes de CO2. Garder les véhicules électriques et hydrogènes risque de biaiser notre modèle. Nous allons donc exclure les véhicules électriques de notre dataset

In [ ]:
# Vérification des émissions de CO2 des véhicules hydrogènes
df_hydrogen = df[df["Carburant"] == "hydrogen"]
df_hydrogen['CO2'].value_counts()

In [ ]:
# Nous excluons les véhicules électriques et hydrogènes
df = df[(df["Carburant"] != "electric") & (df["Carburant"] != "hydrogen")]

In [ ]:
df.shape

# <font color='#3585CD'>Traitement des valeurs manquantes</font>

In [ ]:
data_na = utils.display_missing_values(df)
data_na

## Gestion des valeurs manquantes de la colonne Consommation carburant

In [ ]:
df_fc_na = df[df['Consommation carburant'].isna()]
df_fc_na

Nous allons gérer cette ligne

In [ ]:
# regardons si d'autres modeles RANGE ROVER EVOQUE sont renseignés
df_jag_evoque = df[(df['Modèle'] == 'RANGE ROVER EVOQUE') & (df['Carburant'] == 'diesel') & (~df['Consommation carburant'].isna())]
df_jag_evoque.head()

In [ ]:
fc_mean_jag_evoque = df_jag_evoque['Consommation carburant'].mean().round(1)
fc_mean_jag_evoque

In [ ]:
df.loc[(df["Modèle"] == "RANGE ROVER EVOQUE") & (df['Consommation carburant'].isna()), "Consommation carburant"] = fc_mean_jag_evoque

In [ ]:
data_na = utils.display_missing_values(df)
data_na

## Gestion des valeurs manquantes de la colonne Masse à vide

In [ ]:
data_na = utils.display_missing_values(df)
data_na

In [ ]:
df_mkg_na = df[df['Masse à vide'].isna()]
df_mkg_na

Recherchons dans le dataset si nous avons des modèles équivalents dont les variables m (kg) et Mt ne sont **pas nulles**

In [ ]:
df_jag_lr = df[(df['Modèle'] == 'RANGE ROVER EVOQUE') & (df['Carburant'] == 'diesel') & (df['Puissance moteur'] == 120) & (~df['Masse à vide'].isna())]
df_jag_lr

In [ ]:
fc_mean_masse_jag_evoque = df_jag_evoque['Masse à vide'].mean().round(1)
fc_mean_masse_jag_evoque

In [ ]:
# pour les mêmes modèles, la variable Masse à vide est égale à 1967
df['Masse à vide'] = df['Masse à vide'].fillna(fc_mean_masse_jag_evoque)

In [ ]:
# Vérification
df.loc[23942].to_frame().T

In [ ]:
data_na = utils.display_missing_values(df)
data_na

In [ ]:
# ré inisialisation des indexes
df = df.reset_index(drop=True)
df

# <font color='#3585CD'>Distribution des variables catégorielles</font>

## Analyse par marque

### Valeurs uniques

In [ ]:
sorted(df['Marque'].unique())

### Remplacement

Certaines valeurs peuvent être regroupées, comme par exemple :

*   'MC LAREN', 'MCLAREN'
*   'MERCEDES AMG', 'MERCEDES BENZ', 'MERCEDES-BENZ'
*   'MITSUBISHI', 'MITSUBISHI MOTORS CORPORATION', 'MITSUBISHI MOTORS THAILAND'


In [ ]:
replace_mk = {'MC LAREN' : 'MCLAREN',
              'MERCEDES AMG' : 'MERCEDES BENZ',
              'MERCEDES-BENZ' : 'MERCEDES BENZ',
              'MITSUBISHI MOTORS CORPORATION' : 'MITSUBISHI',
              'MITSUBISHI MOTORS THAILAND' : 'MITSUBISHI',
              'MITSUBISHI MOTORS (THAILAND)' : 'MITSUBISHI',
              'FORD-CNG-TECHNIK' : 'FORD',
              'ROLLS ROYCE' : 'ROLLS-ROYCE'}
df['Marque'] = df['Marque'].replace(replace_mk)

In [ ]:
df['Marque'].nunique()

### Analyse

In [ ]:
utils.analyser_variable_categorielle_plotly(df, 'Marque', 58)

## Analyse par carburant

### Valeurs uniques

In [ ]:
sorted(df['Carburant'].unique())

### Remplacement

Certaines valeurs peuvent être regroupées :



*   'diesel/electric' & 'petrol/electric' sont des véhicules hybride
*   'lpg' & 'ng' sont des énergies au gaz
*   'petrol' sera renommé en 'essence' pour plus de compréhension.


In [ ]:
replace_ft = {
              'petrol' : 'essence',
              'diesel/electric' : 'hybride',
              'petrol/electric' : 'hybride',
              'lpg' : 'gaz',
              'ng' : 'gaz'}
df['Carburant'] = df['Carburant'].replace(replace_ft)

### Analyse

In [ ]:
utils.analyser_variable_categorielle_plotly(df, 'Carburant')

## Analyse par modèle de voiture

In [ ]:
utils.analyser_variable_categorielle_plotly(df, 'Modèle', 30)

# <font color='#3585CD'>Distribution des variables numériques</font>

## Sélection des colonnes numériques

In [ ]:
# Sélection des colonnes numériques
num_vars = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
num_vars

## Analyse de la masse du véhicule Masse à vide

In [ ]:
utils.analyser_variables_numeriques_plotly(df, ['Masse à vide'])

## Analyse des émissions spécifiques de CO2

In [ ]:
utils.analyser_variables_numeriques_plotly(df, ['CO2'], 30)

## Analyse de la cylindrée moteur

In [ ]:
utils.analyser_variables_numeriques_plotly(df, ['Cylindrée moteur'], 50)

## Analyse de la puissance du moteur

In [ ]:
utils.analyser_variables_numeriques_plotly(df, ['Puissance moteur'])

## Analyse de la Consommation carburant

In [ ]:
utils.analyser_variables_numeriques_plotly(df, ['Consommation carburant'])

# <font color='#3585CD'>Création d'indicateurs</font>

## Indicateur de Charge Spécifique du Moteur (ICSM)

`ICSM = Puissance (kW) / Masse du véhicule (kg)`

**Interprétation** :

*   Faible ICSM → Voiture puissante et légère (moins d’effort, moins de CO₂).
*   Élevé ICSM → Voiture sous-motorisée (forte sollicitation, plus de CO₂).

In [ ]:
df['ICSM'] = df['Puissance moteur'] / df['Masse à vide']

## Indicateur de Consommation Énergétique (ICE)

`ICE = Puissance (kW) / Cylindrée (cm³)`

**Interprétation** :

*   Faible ICE → Moteur optimisé (ex. turbo downsizing).
*   Élevé ICE → Moteur gourmand et peu efficient.

In [ ]:
df['ICE'] = df['Puissance moteur'] / df['Cylindrée moteur']

## Indicateur de Densité Energétique du Carburant (IDEC)

`IDEC = Cylindrée (cm³) / Masse du véhicule (kg)`

**Interprétation** :

*   Faible IDEC → Moteur bien dimensionné (moins d’effort, moins de CO₂).
*   Élevé IDEC → Moteur sous-dimensionné (forte sollicitation, plus de CO₂).

In [ ]:
df['IDEC'] = df['Cylindrée moteur'] / df['Masse à vide']

# <font color='#3585CD'>Analyse du CO2 en fonction de certaines variables</font>

### Analyse du CO2 en fonction de la masse du véhicule

In [ ]:
utils.plot_scatter_co2(df, "Masse à vide")

On observe que les véhicules plus lourds tendent à émettre plus de CO₂, avec une distinction entre les types de carburant.

### Analyse du CO2 en fonction de la puissance du moteur.

In [ ]:
utils.plot_scatter_co2(df, "Puissance moteur")

### Analyse du CO2 en fonction du carburant



In [ ]:
utils.plot_scatter_co2(df, "Carburant")

In [ ]:
utils.analyser_hist_co2_par_variable(df, 'Carburant')

### Analyse du CO2 en fonction de la marque

**Top des marques les plus polluantes**

In [ ]:
utils.analyser_hist_co2_par_variable(df, 'Marque', 36)

### Analyse du CO2 en fonction du modèle de voiture

**Top des modèles les plus polluants**

In [ ]:
utils.analyser_hist_co2_par_variable(df, 'Modèle', 30)

**Top des modèles les moins polluants**

In [ ]:
utils.analyser_hist_co2_par_variable(df, 'Modèle', 30, order='asc')

### Analyse du CO2 en fonction de la consommation

In [ ]:
utils.plot_scatter_co2(df, "Consommation carburant")

### Analyse du CO2 en fonction des indicateurs ICSM, ICE, IDEC

In [ ]:
utils.plot_scatter_co2(df, "ICSM")

In [ ]:
utils.plot_scatter_co2(df, "ICE")

In [ ]:
utils.plot_scatter_co2(df, "IDEC")

# <font color='#3585CD'>Corrélation des variables numériques</font>

In [ ]:
utils.plot_correlation_matrix(df)

In [ ]:
num_numeric_cols = df.select_dtypes(include=['number']).columns
num_numeric_cols

for col in num_numeric_cols:
  utils.calculer_correlation(df, col, 'CO2')

# <font color='#3585CD'>Analyse des outliers</font>

## Masse à vide

In [ ]:
utils.detecter_outliers_plotly_var(df['Masse à vide'])

In [ ]:
df_outliers_masse = df[df['Masse à vide'] < 750].sort_values(by='Masse à vide', ascending=True)
df_outliers_masse.head(20)

In [ ]:
df_outliers_masse = df[df['Masse à vide'] > 2500].sort_values(by='Masse à vide', ascending=True)
df_outliers_masse.tail(20)

## Cylindrée moteur

In [ ]:
utils.detecter_outliers_plotly_var(df['Cylindrée moteur'])

In [ ]:
df_outliers_CO2 = df[df['Cylindrée moteur'] > 4000].sort_values(by='Cylindrée moteur', ascending=True)
df_outliers_CO2.tail(20)

## CO2

In [ ]:
utils.detecter_outliers_plotly_var(df['CO2'])

In [ ]:
df_outliers_CO2 = df[df['CO2'] > 400].sort_values(by='CO2', ascending=True)
df_outliers_CO2.tail(20)

## Puissance moteur

In [ ]:
utils.detecter_outliers_plotly_var(df['Puissance moteur'])

In [ ]:
df_outliers_Consommation = df[df['Puissance moteur'] > 15].sort_values(by='Puissance moteur', ascending=True)
df_outliers_Consommation.tail(20)

## Consommation carburant

In [ ]:
utils.detecter_outliers_plotly_var(df['Consommation carburant'])

In [ ]:
df_outliers_Consommation = df[df['Consommation carburant'] > 15].sort_values(by='Consommation carburant', ascending=True)
df_outliers_Consommation.tail(20)

# <font color='#3585CD'>Visualisation globale graphique</font>

In [ ]:
fig = px.scatter_matrix(df, dimensions=df.select_dtypes(include=['number']).columns,
                        color='Carburant', title="Pairplot")

fig.update_layout(height=900, width=1200)
fig.show()

In [ ]:
utils.plot_correlation_matrix(df)

# <font color='#3585CD'>Distribution de la variable cible</font>





## Histogramme et boxplot de la variable cible

In [ ]:
utils.analyser_variables_numeriques_plotly(df, ['CO2'])

In [ ]:
import plotly.graph_objects as go
import numpy as np
import scipy.stats as stats
import pandas as pd

# Calcul des quantiles
(quantiles, values), (slope, intercept, r) = stats.probplot(df['CO2'], dist="norm")

# Création du Q-Q Plot avec Plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=quantiles, y=values, mode='markers', name='Données'))
fig.add_trace(go.Scatter(x=quantiles, y=slope * np.array(quantiles) + intercept, mode='lines', name='Ligne théorique'))

fig.update_layout(title='Q-Q plot pour CO2', xaxis_title='Quantiles théoriques', yaxis_title='Quantiles des données')

fig.show()

In [ ]:
from statsmodels.stats.diagnostic import lilliefors
# Niveau de signification (alpha)
alpha = 0.05

# Effectuer les tests de normalité
shapiro_test = stats.shapiro(df['CO2'])
ks_test = stats.kstest(df['CO2'], 'norm')
ad_test = stats.anderson(df['CO2'], dist='norm')
dagostino_test = stats.normaltest(df['CO2'])
lilliefors_test = lilliefors(df['CO2'], dist='norm')

# Créer un tableau pandas avec les résultats des tests
test_results = pd.DataFrame({
    'Nom du test': ['Shapiro-Wilk', 'Kolmogorov-Smirnov', 'Anderson-Darling', "D'Agostino-Pearson", 'Lilliefors'],
    'Statistique de test': [shapiro_test[0], ks_test.statistic, ad_test.statistic, dagostino_test.statistic, lilliefors_test[0]],
    'p-valeur': [shapiro_test[1], ks_test.pvalue, None, dagostino_test.pvalue, lilliefors_test[1]],
    'Normalité': ['Oui' if shapiro_test[1] > alpha else 'Non',
                  'Oui' if ks_test.pvalue > alpha else 'Non',
                  'Oui' if any(ad_test.statistic < crit_val for crit_val in ad_test.critical_values) else 'Non',
                  'Oui' if dagostino_test.pvalue > alpha else 'Non',
                  'Oui' if lilliefors_test[1] > alpha else 'Non']
})

print(test_results)


In [ ]:
# from scipy.stats import boxcox

# df['co2_transformed'], lambda_boxcox = boxcox(df['Ewltp (g/km)'] + 1)  # Ajouter 1 pour éviter 0
# df

In [ ]:
df['Carburant'].unique()

In [ ]:
# Histogramme en fonction du type de motorisation
fig = px.histogram(df, x="CO2", color="Carburant", nbins=50, barmode="overlay",
                   title="Distribution des émissions de CO2 par type de carburant")
fig.show()


Comment gérer ?


*   Création colonne Hybride ?
*   Dataset à part ?
*   Juste un OneHotEncoder ?

## Degrés d'asymétrie des variables

In [ ]:
num_vars = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
df[num_vars].skew().sort_values()

### CO2

Légère asymétrie négative (peu inquiétante)

### Consommation carburant

 Quasi symétrique (très léger)

### Masse moyenne

Asymétrie positive modérée à forte, probablement due à des valeurs élevées qui tirent la distribution.

# <font color='#3585CD'>Quelles variables garder pour prédire le CO2 ?</font>

Selon nous, les **variables pertinentes** à garder pour la prédiction sont les suivantes :
* **Masse à vide** : un véhicule plus lourd a souvent des émissions plus élevées.
* **Cylindrée moteur** : une plus grande cylindrée est souvent associée à une plus grande consommation et donc plus d’émissions.
* **Puissance moteur** : un moteur plus puissant a tendance à consommer plus de carburant.
* **Carburant** : Essence, diesel, hybride… Chaque type influence les émissions. La variable Carburant sera encodée.

* **Consommation carburant** : directement liée aux émissions de CO2. Cependant sa forte corrélation peut expliquer à elle seule les émissions de CO2. Nous nous questionnerons au moment de la modélisation si nous la gardons ou pas.

Nous **excluerons** les variables **Marque** et **Modèle** qui ne sont pas directement liées aux émissions de CO2.

Pourquoi exclure les variables **Marque** et **Modèle** ?
* Ce sont des variables catégoriques à très haute **cardinalité**
* L'influence sur le CO₂ passe par des **caractéristiques techniques**

# <font color='#3585CD'>Export du dataset nettoyé</font>

In [ ]:
df.shape

In [ ]:
df = df.reset_index(drop=True)
df

In [ ]:
# Vérifier si on est sur Google Colab
import os
try:
    import google.colab
    ON_COLAB = True
    dataset_path = "/content/drive/My Drive/Formation DS/Projet CO2/NOV24-CDS-CO2/notebooks/datasets/Dataset_final/datas_nettoyees_model_FR.csv"

    # Monter Google Drive si ce n'est pas déjà fait
    from google.colab import drive
    drive.mount('/content/drive')

except ImportError:
    ON_COLAB = False
    dataset_path = "datasets/Dataset_final/datas_nettoyees_model_FR.csv"

# Sauvegarde du DataFrame
df.to_csv(dataset_path, index=False)

# Vérification de l'enregistrement
if os.path.exists(dataset_path):
    print(f"Le fichier a bien été enregistré à l'emplacement : {dataset_path}")
else:
    print("Problème lors de l'enregistrement du fichier.")


In [ ]:
# dataset_path = "datasets/Dataset_final/datas_nettoyees_model_FR.csv"
# df.to_csv(dataset_path, index =False)

# import os

# # Vérifie si le fichier existe
# if os.path.exists(dataset_path):
#     print("Le fichier a bien été enregistré.")
# else:
#     print("Problème lors de l'enregistrement du fichier.")
